In [48]:
import requests as req
import pandas as pd

In [102]:
WYCO_BASE_URL = 'http://ils.wycosheriff.org/InmateService.svc'

Jail = 'WCDC2'
Fields = 'ABCDEFGHIJMNPQRST1235'
isLogin = 'false'

def gen_key():
    resp_gen_pass_key = req.get(WYCO_BASE_URL + '/GeneratePassKey')
    pass_key_result = resp_gen_pass_key.json()['GeneratePassKeyResult']
    return pass_key_result.split('|||')

def get_inmates(key, ans):
    resp_inmates = req.get(WYCO_BASE_URL+'/GetInmates2?Jail='+Jail+'&&key='+key+'&ans='+ans)
    inmates_result = resp_inmates.json()['GetInmates2Result']
    df_inmates = pd.DataFrame(inmates_result)
    df_inmates.to_csv(Jail+'_inmates.csv')
    return df_inmates

# set load_count = 0 to load all
def get_inmates_charges(df_inmates, key, ans, load_count):
    df_charges = pd.DataFrame()
    i = 0
    for BookNo in df_inmates['BookNo']:
        resp_charges = req.get(WYCO_BASE_URL+'/GetInmateCharges?Jail='+Jail+'&&bookno='+BookNo+'&Fields='+Fields+'&isLogin='+isLogin)
        charges_result = resp_charges.json()['GetInmateChargesResult']
        df = pd.DataFrame(charges_result)
        df['BookNo'] = BookNo
        df_charges = df_charges.append(df)
        
        i = i + 1
        
        if i % 20 == 0:
            print('Loaded charges '+str(i)+' of '+str(len(df_inmates.index)))
            
        if load_count != 0 and i > load_count:
            break;
    
    print('Done')
    df_charges.to_csv(Jail+'_charges.csv', index=False)
    return df_charges
    
#r = req.get('http://ils.wycosheriff.org/InmateService.svc/GetInmate?Jail='+Jail+'&&bookno='+BookNo+'&key='+Key+'&answer='+Ans+'&Fields='+Fields+'&isLogin='+isLogin)

In [103]:
(key, ans) = gen_key()
df_inmates = get_inmates(key, ans)
df_charges = get_inmates_charges(df_inmates, key, ans, 0)

df_inmates_charges = df_inmates.merge(df_charges, on='BookNo', how='left')
df_inmates_charges.to_csv(Jail+'_inmates_charges.csv')

Loaded 20 of 417
Loaded 40 of 417
Loaded 60 of 417
Loaded 80 of 417
Loaded 100 of 417
Loaded 120 of 417
Loaded 140 of 417
Loaded 160 of 417
Loaded 180 of 417
Loaded 200 of 417
Loaded 220 of 417
Loaded 240 of 417
Loaded 260 of 417
Loaded 280 of 417
Loaded 300 of 417
Loaded 320 of 417
Loaded 340 of 417
Loaded 360 of 417
Loaded 380 of 417
Loaded 400 of 417
Done
